## STORE SALES TIMESERIES FORECAST

### TABLE OF CONTENTS

A. **Pace: Plan stage**

    1. Introduction: Definition of forecasting task with identification of features and target

B. **pAce: Analyze stage**

    2. Data Preparation
    3. Data Visualization
    4. Multistep forecasting strategies: multioutput, direct, recursive, DiRec

C. **paCe: Construct stage**

    5. Model Construction: Boosted Hybrid vs Stacked hybrid

D. **pacE: Execute stage**

    6. Hyperparameter tuning
    7. Conclusion

### **A. Pace: Plan Stage**

1. Introduction: Definition of forecasting task with identification of features and target
    In this section, we will define the forecasting task and identify the relevant features and target variable.

#### Introduction

This project aims to forecast store sales using a time series dataset from Kaggle. The primary goal is to predict future sales for each product family sold at Favorita stores located in Ecuador. Accurate forecasts can help in inventory management, staffing, and overall business strategy.


#### Identification of Features and Target

Features:
- Store ID
- Date
- Promotion
- Holiday
- Day of the week
- Season
- Weather

Target:
- Sales 

In [1]:
%%time

## Imports

# Installing select libraries

# General library imports
from gc import collect
from warnings import filterwarnings
filterwarnings('ignore')
from IPython.display import clear_output

# Data manipulation and visualization
import pandas as pd
import numpy as np
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline

# Model and pipeline specifics
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_log_error 
from sklearn.model_selection import KFold, train_test_split

# Time series specific
from statsmodels.tsa.deterministic import DeterministicProcess

from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Ensemble and tuning
import optuna
from optuna import create_study
optuna.logging.set_verbosity(optuna.logging.ERROR)

CPU times: total: 1.72 s
Wall time: 6.2 s


In [2]:
# Setting rc parameters in seaborn for plots and graphs
sns.set({"axes.facecolor": "#f7f9fc",
          "figure.facecolor": "#f7f9fc",
          "axes.edgecolor": "#000000",
          "grid.color": "#EBEBE7",
          "font.family": "serif",
          "axes.labelcolor": "#000000",
          "xtick.color": "#000000",
          "ytick.color": "#000000",
          "grid.alpha": 0.4,
         "grid.linewidth"       : 0.75,
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : '#0099e6',
         'axes.titlesize'       : 8.5,
         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,
        })

# Making sklearn pipeline outputs as dataframe
from sklearn import set_config
set_config(transform_output = "pandas")
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.2f}'.format

print()
collect()

20

### **B. pAce: Analyze Stage**

#### Data Preparation

In [3]:
## loading the sales by store dataset: train

store_sales = pd.read_csv(
    'data/train.csv',
    usecols=[
        'store_nbr', 'date', 'family', 'sales', 'onpromotion'
    ],
    dtype={
        'family': 'category',
        'store_nbr': 'category',
        'sales': 'float16', 
        'onpromotion': 'float16',
    }, 
    parse_dates=['date'],
    infer_datetime_format=True,
)

# Verify that all columns are loaded
print(store_sales.head())
print(store_sales.dtypes)

        date store_nbr      family  sales  onpromotion
0 2013-01-01         1  AUTOMOTIVE   0.00         0.00
1 2013-01-01         1   BABY CARE   0.00         0.00
2 2013-01-01         1      BEAUTY   0.00         0.00
3 2013-01-01         1   BEVERAGES   0.00         0.00
4 2013-01-01         1       BOOKS   0.00         0.00
date           datetime64[ns]
store_nbr            category
family               category
sales                 float16
onpromotion           float16
dtype: object


In [4]:
## loading the sales by store dataset: train

store_sales_test = pd.read_csv(
    'data/test.csv',
    usecols=[
        'store_nbr', 'date', 'family', 'onpromotion'
    ],
    dtype={
        'family': 'category',
        'store_nbr': 'category', 
        'onpromotion': 'float16',
    }, 
    parse_dates=['date'],
    infer_datetime_format=True,
)

# Verify that all columns are loaded
print(store_sales_test.head())

        date store_nbr      family  onpromotion
0 2017-08-16         1  AUTOMOTIVE         0.00
1 2017-08-16         1   BABY CARE         0.00
2 2017-08-16         1      BEAUTY         2.00
3 2017-08-16         1   BEVERAGES        20.00
4 2017-08-16         1       BOOKS         0.00


In [5]:
## this is an additional dataset containing the number of transactions that held each day;

store_transaction = pd.read_csv(
    'data/transactions.csv',
    usecols=[
        'date', 'store_nbr', 'transactions'
    ],
    dtype={
        'store_nbr': 'category',
        'transactions': 'float16', 
    }, 
    parse_dates=['date'],
    infer_datetime_format=True,
)

In [6]:
store_transaction = store_transaction.groupby(['date', 'store_nbr']).mean()

In [7]:
## merging the transaction and the sales per store dataset

sales_transaction_per_store = store_sales.merge(
    store_transaction,
    on=['date','store_nbr'],
    how='left'
)

# print(sales_transaction_per_store)

## merging the transaction and the test sales per store dataset
sales_transaction_test_per_store = store_sales_test.merge(
    store_transaction,
    on=['date','store_nbr'],
    how='left'
)

In [8]:
## loading the stores dataset

stores = pd.read_csv(
    'data/stores.csv',
    usecols=[
        'store_nbr', 'city', 'cluster'
    ],
    dtype={
        'store_nbr' : 'category',
        'city': 'category',
        'cluster': 'int64',
    },
)

In [9]:
## merged the sales and transactions per store dataset with the store dataset;

stores_added = sales_transaction_per_store.merge(
    stores, 
    on = 'store_nbr',
    how = 'left'
)

# print(stores_added)

## merged the test sales and transactions per store datset with the store dataset;

stores_added_test = sales_transaction_test_per_store.merge(
    stores, 
    on ='store_nbr',
    how = 'left'
)

In [10]:
## loaded the oil dataset with the existing dataset to assess the impact of oil
## on the sales among product families as Ecuador is an oil-producing country and its economy 
## depends heavily on it.

oil = pd.read_csv(
    'data/oil.csv',
    usecols=[
        'date', 'dcoilwtico'
    ],
    dtype={
        'dcoilwtico': 'float32', 
    }, 
    parse_dates=['date'],
    infer_datetime_format=True,
)

In [11]:
## next, we merge the dataset with the existing dataset;

oil_added = stores_added.merge(
    oil, 
    on = 'date',
    how = 'left'
)

# print(oil_added)

## merge the dataset with the existing test dataset too

oil_added_test = stores_added_test.merge(
    oil, 
    on = 'date',
    how = 'left'
)

In [12]:
## loading the holidays dataset to see the effects of holidays on the sales of products;

holidays = pd.read_csv(
    'data/holidays_events.csv',
    usecols=[
        'date', 'type', 'locale', 'description', 'transferred'
    ],
    dtype={
        'type': 'category',
        'locale': 'category',
        'description' : 'category', 
        'transferred': 'bool', 
    }, 
    parse_dates=['date'],
    infer_datetime_format=True,
)

In [13]:
## creating a corresponding matching holidays dataset for the test dataset

# Define the date range
start_date = '2016-08-16'
end_date = '2016-08-31'

# Filter the DataFrame to extract rows within the date range
filtered_holidays = holidays[(holidays['date'] >= start_date) & (holidays['date'] <= end_date)]

# Change the year from 2016 to 2017 for all filtered dates
filtered_holidays['date'] = filtered_holidays['date'].apply(lambda x: x.replace(year=2017))

# Display the updated DataFrame
print(filtered_holidays)

          date     type locale          description  transferred
271 2017-08-24  Holiday  Local  Fundacion de Ambato        False


In [14]:
## merged the holidays dataset with the pre-existing dataset

holidays_added = oil_added.merge(
    holidays, 
    on = 'date',
    how = 'left'
)

# print(holidays_added)

## merge the holidays dataset with the pre-existing test dataset

filtered_holidays['date'] = pd.to_datetime(filtered_holidays['date'])

holidays_added_test = oil_added_test.merge(
    filtered_holidays, 
    on = 'date',
    how = 'left'
)

In [15]:
## the final complex dataset;

store_sales_final = holidays_added
store_sales_test = holidays_added_test

In [16]:
# Handling missing values for the training dataset

# Handle categorical variables
categorical_vars = ['type', 'locale', 'description']
for var in categorical_vars:
    # Convert categorical columns to object type
    if pd.api.types.is_categorical_dtype(store_sales_final[var]):
        store_sales_final[var] = store_sales_final[var].astype(str)
    store_sales_final[var].fillna('unknown', inplace=True)
    # Optionally convert back to categorical type
    # store_sales_final[var] = store_sales_final[var].astype('category')

# Handle numerical variables
numerical_vars = ['transactions', 'dcoilwtico']
for var in numerical_vars:
    store_sales_final[var].fillna(store_sales_final[var].mean(), inplace=True)

# Handle boolean variables
boolean_var = 'transferred'
mode_transferred = store_sales_final[boolean_var].mode()[0]
store_sales_final[boolean_var].fillna(mode_transferred, inplace=True)

# Handling missing values for the test dataset

# Handle categorical variables
for var in categorical_vars:
    # Convert categorical columns to object type
    if pd.api.types.is_categorical_dtype(store_sales_test[var]):
        store_sales_test[var] = store_sales_test[var].astype(str)
    store_sales_test[var].fillna('unknown', inplace=True)
    # Optionally convert back to categorical type
    # store_sales_test[var] = store_sales_test[var].astype('category')

# Handle numerical variables
for var in numerical_vars:
    store_sales_test[var].fillna(store_sales_final[var].mean(), inplace=True)  # Use training data mean for consistency

# Handle boolean variables
store_sales_test[boolean_var].fillna(mode_transferred, inplace=True)  # Use training data mode for consistency

In [17]:
# store_sales_final.fillna('0', inplace=True)
store_sales_test['transactions'].fillna('0', inplace=True)

In [18]:
# Convert 'date' columns to period type
store_sales_final['date'] = store_sales_final['date'].dt.to_period('D')
store_sales_test['date'] = store_sales_test['date'].dt.to_period('D')

# Set index and sort
store_sales_final = store_sales_final.set_index(['store_nbr', 'family', 'date', 'city', 'locale', 'type', 'description']).sort_index()
store_sales_test = store_sales_test.set_index(['store_nbr', 'family', 'date', 'city', 'locale', 'type', 'description']).sort_index()

# Ensure numerical columns are properly typed
numeric_cols = ['sales', 'onpromotion', 'transactions', 'dcoilwtico']
store_sales_final[numeric_cols] = store_sales_final[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Only include columns in the test dataset that are present
test_numeric_cols = [col for col in numeric_cols if col in store_sales_test.columns]
store_sales_test[test_numeric_cols] = store_sales_test[test_numeric_cols].apply(pd.to_numeric, errors='coerce')

# Extract recent sales from 2016-2017 for the training and validation dataset
family_sales = (
    store_sales_final.groupby(['family', 'date'])
    .mean()  # Compute the mean for numeric columns
    .unstack('family')
    .loc['2016':'2017']  # Filter for years 2016 and 2017
)

# Extract recent sales from 2017 for the test dataset
family_sales_test = (
    store_sales_test.groupby(['family', 'date'])
    .mean()  # Compute the mean for numeric columns present in the test dataset
    .unstack('family')
    .loc['2017']  # Filter for the year 2017
)

In [19]:
## Handling missing values by filling; once tried nearest neighbors filling - also known as interpolation

family_sales = family_sales.fillna(family_sales.mean())

In [20]:
## handling duplicate values by dropping them

family_sales = family_sales.drop_duplicates()
family_sales_test = family_sales_test.drop_duplicates()

In [21]:
# train dataset;

# Dataset preprocessing for the model construction;

## Target series
y = family_sales.loc[:, 'sales']

# X1: Features for Linear Model
dp = DeterministicProcess(index=y.index, order=1)
X1_train = dp.in_sample()

# X2: Features for Ensemble model
X2_train = family_sales.drop('sales', axis=1) # onpromotion feature, store feature, 

X2_train = pd.get_dummies(X2_train, drop_first=True) #

# test dataset;

# Dataset preprocessing for the model construction;

# X1: Features for Linear Model
dp = DeterministicProcess(index=family_sales_test.index, order=1)
X1_test = dp.in_sample()
# print(X1_test)

# X2: Features for Ensemble model
X2_test = family_sales_test # onpromotion feature, store feature, 

X2_test = pd.get_dummies(X2_test, drop_first=True) #

In [22]:
# Define the cutoff date for the split
cutoff_date = '2017-05-31'

# Split the data based on the cutoff date
y_train = y.loc[:cutoff_date]
y_valid = y.loc[cutoff_date:]

X1_valid = X1_train.loc[cutoff_date:]
X1_train = X1_train.loc[:cutoff_date]

X2_valid = X2_train.loc[cutoff_date:]
X2_train = X2_train.loc[:cutoff_date]

In [23]:
# Clean the target variable
if np.any(np.isinf(y_train)):
    y_train.replace([np.inf, -np.inf], np.nan, inplace=True)

if y_train.isna().any().any():
    y_train.fillna(0, inplace=True)

y_train = y_train.astype(np.float64)

### **C. paCe: Construct Stage**

In [24]:
# Define your model combinations
models_1 = [LinearRegression(), ElasticNet(), Lasso(), Ridge()]
models_2 = [RandomForestRegressor(), KNeighborsRegressor(), XGBRegressor()]

#### BoostedHybrid class creation

In [25]:
class boostedHybrid():
    def __init__(self, model_1, model_2): 
        self.model_1 = model_1
        self.model_2 = model_2
        self.y_is_series = False  # To check if y is a Series or DataFrame
        self.y_columns = None  # To store the columns of y if it's a DataFrame
    
    def fit(self, X_1, X_2, y):
        # Check if y is a Series or DataFrame and handle accordingly
        if isinstance(y, pd.Series):
            self.y_is_series = True
            y = y.to_frame()
        
        # Save columns for later use in predict method
        self.y_columns = y.columns
        
        # Fit model_1
        self.model_1.fit(X_1, y)
        
        y_fit = pd.DataFrame(self.model_1.predict(X_1), index=X_1.index, columns=self.y_columns)

        # Compute residuals
        y_resid = y - y_fit

        # Fill NaNs in residuals
        y_resid.fillna(0, inplace=True)

        # Fit model_2 on residuals
        self.model_2.fit(X_2, y_resid)

        return y_fit # this is so that we can plot the y fitted values on a graph
    
    def predict(self, X_1, X_2):
        y_pred = pd.DataFrame(self.model_1.predict(X_1), index=X_1.index, columns=self.y_columns)
        
        # Predict residuals using model_2
        y_resid_pred = self.model_2.predict(X_2)
        
        # Ensure that y_resid_pred has the same shape as y_pred
        if len(y_pred.shape) == 1:
            y_resid_pred = pd.Series(y_resid_pred, index=X_1.index)
            y_pred = y_pred.add(y_resid_pred, axis=0)
        else:
            y_resid_pred = pd.DataFrame(y_resid_pred, index=X_1.index, columns=self.y_columns)
            y_pred = y_pred.add(y_resid_pred, axis=1)
        
        # If original y was a Series, return a Series
        if self.y_is_series:
            return y_pred.squeeze()
        
        return y_pred

### **D. pacE: Execute stage**

#### Model selection based on least root mean square logarithmic error

In [26]:
# Initialize variables to store the best model and its RMSLE
best_model = None
best_rmsle = float('inf')
best_y_fit = None
best_y_pred = None

# Iterate through all combinations of model_1 and model_2
for model_1 in models_1:
    for model_2 in models_2:
        # Create the boosted hybrid model
        model = boostedHybrid(model_1, model_2)

        # Fit the model
        y_fit = model.fit(X1_train, X2_train, y_train)

        # Predict
        y_pred = model.predict(X1_train, X2_train)

        # Clip the predicted values and actual values to be non-negative
        y_pred_clipped = y_pred.clip(lower=0)
        y_train_clipped = y_train.clip(lower=0)

        # Calculate RMSLE
        rmsle_score = np.sqrt(mean_squared_log_error(y_train_clipped, y_pred_clipped))

        # Update the best model and its RMSLE if a better model is found
        if rmsle_score < best_rmsle:
            best_model = model
            best_rmsle = rmsle_score
            best_y_fit = y_fit
            best_y_pred = y_pred

# Print the best model and its RMSLE
print("Best Model:", best_model)
print("Best RMSLE:", best_rmsle)

Best Model: <__main__.boostedHybrid object at 0x000001E3F1FF62A0>
Best RMSLE: 0.019229230244921593


##### Best model is a combination of the Ridge and XGBoost as the linear and ensemble models respectively

In [27]:
best_model.model_1

LinearRegression()

In [28]:
best_model.model_2

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

### save model

In [36]:
joblib.dump(best_model, 'model/boostedhybrid.pkl')

['model/boostedhybrid.pkl']

#### Using the best model to forecast sales over the year 2017

In [29]:
y_pred_valid = best_model.predict(X1_valid, X2_valid)
y_pred_test = best_model.predict(X1_test, X2_test)

In [30]:
# Concatenate predictions
y_pred = pd.concat([y_pred_valid, y_pred_test])

# Sort by index if needed
y_pred = y_pred.sort_index()

#### Data Visualization 

In [31]:
# creating visuals for all product families

families = y.columns

In [32]:
# Check if index is a PeriodIndex before converting
if isinstance(y.index, pd.PeriodIndex):
    y.index = y.index.to_timestamp()

if isinstance(y_fit.index, pd.PeriodIndex):
    y_fit.index = y_fit.index.to_timestamp()

if isinstance(y_pred.index, pd.PeriodIndex):
    y_pred.index = y_pred.index.to_timestamp()

In [34]:
# List of improved colors
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Blue, Orange, Green

# Start date for filtering the plots
start_date = pd.Timestamp('2017-06-01')

# Iterate through each family and create a plot
for i, family in enumerate(families):
    # Create a new figure for each family
    fig = go.Figure()

    # Filter data starting from the start_date
    y_fit_filtered = y_fit.loc[y_fit.index >= start_date]
    y_filtered = y.loc[y.index >= start_date]
    y_pred_filtered = y_pred.loc[y_pred.index >= start_date]

    # Add fitted data trace
    fig.add_trace(go.Scatter(
        x=y_fit_filtered.index, 
        y=y_fit_filtered[family], 
        mode='lines', 
        line=dict(color=colors[0], dash='dash', width=1), 
        name=f'{family} Fitted'
    ))

    # Add original data trace
    fig.add_trace(go.Scatter(
        x=y_filtered.index, 
        y=y_filtered[family], 
        mode='lines', 
        line=dict(color=colors[1], width=1),  # Slightly thicker line for clarity
        name=f'{family} Original'
    ))

    # Add predicted data trace
    fig.add_trace(go.Scatter(
        x=y_pred_filtered.index, 
        y=y_pred_filtered[family], 
        mode='lines', 
        line=dict(color=colors[2], dash='dot', width=1), 
        name=f'{family} Predicted'
    ))

    # Update layout with interactive filter options
    fig.update_layout(
        title=f'Original, Fitted, and Predicted Sales for {family}',
        xaxis_title='Date',
        yaxis_title='Sales',
        template='plotly_dark',  # Use the dark theme as a base
        plot_bgcolor='#303030',  # Slightly darker background color
        paper_bgcolor='#2d2d2d',  # Darker paper background
        updatemenus=[
            dict(
                active=0,
                buttons=[
                    dict(label="Show All",
                         method="update",
                         args=[{"visible": [True, True, True]},
                               {"title": f"All Data for {family}"}]),
                    dict(label="Fitted Only",
                         method="update",
                         args=[{"visible": [False, True, False]},
                               {"title": f"Fitted Sales for {family}"}]),
                    dict(label="Predicted Only",
                         method="update",
                         args=[{"visible": [False, False, True]},
                               {"title": f"Predicted Sales for {family}"}])
                ],
                direction="down",
                showactive=True,
            )
        ],
        legend=dict(
            title='',  # Remove the legend title
            orientation="h",
            yanchor="bottom",
            y=1.05,  # Further lowered legend for more spacing
            xanchor="center",
            x=0.5,
            font=dict(
                size=10.5  # Slightly larger font size for the legend
            )
        ),
        title_font=dict(
            size=16  # Title font size
        ),
        font=dict(
            family="Arial, sans-serif",
            size=12,
            color="#ffffff"  # Font color adjusted for dark background
        ),
        margin=dict(l=20, r=20, t=100, b=40),  # Further increased top margin
        width=660,  # Plot width
        height=280,  # Plot height
        xaxis=dict(
            range=[start_date, None]  # Set the start date and leave end date as None
        )
    )

    # Show the plot 
    fig.show()

##### Thank you for staying through to the end. I'm Ikechukwu Ugbo, a final-year medical student, and researcher at the University of Ibadan and to you a data scientist analyst.